In [58]:
import requests
import pandas as pd
import numpy as np
import ast

from datetime import datetime, timedelta
pd.options.display.max_columns = None

Usando el csv de "attacks_limpieza_completa y el csv con los datos climáticos del anterior pair, deberemos juntarlos en una única fuente de información.



In [59]:
df_tiburones = pd.read_csv("datos/attacks_limpieza_completa.csv", index_col=0)

df_clima = pd.read_csv("datos/tiempo_paises_1.csv", index_col=0).reset_index()

In [60]:
df_clima.head(2)

,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,0,3,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",4,15,10,1015,none,0,0,235,2,USA
1,1,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",4,15,8,1014,none,0,0,180,3,USA


In [61]:
df_tiburones.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860


El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. 
Para ello:
- Cargaremos los dos ficheros de datos
- Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer: (USA, Australia, New Zealand, South Africa, Papua New Guinea)
- Del dataframe de los datos climáticos seleccionaremos todas las columnas.
- Cuando ya tengamos todos los datos deseados juntaremos los dos csv.
- Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.
- Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. 
Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

In [62]:
paises = ["usa", "australia", "new zealand", "south africa", "papua new guinea"]

In [63]:
df_tiburones_country =df_tiburones[df_tiburones["country"].isin(paises)]
df_tiburones_country["country"].unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

In [64]:
# Como los datos de las columnas, respecto a los paises, en un df está en capitalize y en el otro en minúsculas vamos a unificarlo un poco.

df_clima["country"] = df_clima["country"].str.lower()

In [65]:
df_clima.head(1)

,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,0,3,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",4,15,10,1015,none,0,0,235,2,usa


In [66]:
df_tiburones_country.head(1)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327


Procedemos a limpiar las columnas del df_clima de "rh_profile" y "wind_profile" 

In [67]:
df_clima['rh_profile'] = df_clima['rh_profile'].apply(ast.literal_eval)

In [68]:
df_rh = df_clima["rh_profile"].apply(pd.Series)
df_rh.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': -9999}","{'layer': '900mb', 'rh': 3}","{'layer': '850mb', 'rh': 2}","{'layer': '800mb', 'rh': 2}","{'layer': '750mb', 'rh': 3}","{'layer': '700mb', 'rh': 5}","{'layer': '650mb', 'rh': 4}","{'layer': '600mb', 'rh': 2}","{'layer': '550mb', 'rh': 11}","{'layer': '500mb', 'rh': 14}","{'layer': '450mb', 'rh': 9}","{'layer': '400mb', 'rh': 6}","{'layer': '350mb', 'rh': 8}","{'layer': '300mb', 'rh': 13}","{'layer': '250mb', 'rh': 11}","{'layer': '200mb', 'rh': 3}"


In [69]:
for i in range(len(df_rh.columns)):
    
    nombre = "rh_" + str(df_rh[i].apply(pd.Series)["layer"][0]) 

    #print(nombre)

    valores = list(df_rh[i].apply(pd.Series)["rh"])

    #print(valores)

    df_clima.insert(i, nombre, valores)
    


In [70]:
df_clima.head(1)

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,-9999,3,2,2,3,5,4,2,11,14,9,6,8,13,11,3,0,3,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",4,15,10,1015,none,0,0,235,2,usa


Hacemos lo mismo con la columna de wind_profile:

In [71]:
df_clima['wind_profile']= df_clima['wind_profile'].apply(ast.literal_eval)

In [72]:
df_wind= df_clima["wind_profile"].apply(pd.Series)
df_wind.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'direction': -9999, 'speed'...","{'layer': '900mb', 'direction': 305, 'speed': 3}","{'layer': '850mb', 'direction': 295, 'speed': 3}","{'layer': '800mb', 'direction': 275, 'speed': 5}","{'layer': '750mb', 'direction': 285, 'speed': 5}","{'layer': '700mb', 'direction': 300, 'speed': 6}","{'layer': '650mb', 'direction': 285, 'speed': 6}","{'layer': '600mb', 'direction': 275, 'speed': 5}","{'layer': '550mb', 'direction': 285, 'speed': 6}","{'layer': '500mb', 'direction': 295, 'speed': 7}","{'layer': '450mb', 'direction': 295, 'speed': 7}","{'layer': '400mb', 'direction': 305, 'speed': 6}","{'layer': '350mb', 'direction': 320, 'speed': 9}","{'layer': '300mb', 'direction': 315, 'speed': 10}","{'layer': '250mb', 'direction': 315, 'speed': 12}","{'layer': '200mb', 'direction': 315, 'speed': 7}"


In [73]:
for i in range(len(df_wind.columns)):

    nombre = "wind_direction" + str(df_wind[i].apply(pd.Series)["layer"][0])
    
    #print(nombre)

    valor1 = list(df_wind[i].apply(pd.Series)["direction"])
    
    #print(valores)
    
    df_clima.insert(i, nombre, valor1)


for i in range(len(df_wind.columns)):

    nombre2 = "wind_speed" + str(df_wind[i].apply(pd.Series)["layer"][0]) 
    #print(nombre)

    valor2 = list(df_wind[i].apply(pd.Series)["speed"])

    #print(valores)
      
    df_clima.insert(i, nombre2, valor2)

In [74]:
df_clima.head(2)

,wind_speed950mb,wind_speed900mb,wind_speed850mb,wind_speed800mb,wind_speed750mb,wind_speed700mb,wind_speed650mb,wind_speed600mb,wind_speed550mb,wind_speed500mb,wind_speed450mb,wind_speed400mb,wind_speed350mb,wind_speed300mb,wind_speed250mb,wind_speed200mb,wind_direction950mb,wind_direction900mb,wind_direction850mb,wind_direction800mb,wind_direction750mb,wind_direction700mb,wind_direction650mb,wind_direction600mb,wind_direction550mb,wind_direction500mb,wind_direction450mb,wind_direction400mb,wind_direction350mb,wind_direction300mb,wind_direction250mb,wind_direction200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,-9999,3,3,5,5,6,6,5,6,7,7,6,9,10,12,7,-9999,305,295,275,285,300,285,275,285,295,295,305,320,315,315,315,-9999,3,2,2,3,5,4,2,11,14,9,6,8,13,11,3,0,3,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",4,15,10,1015,none,0,0,235,2,usa
1,-9999,3,4,4,4,5,5,6,6,6,6,6,7,9,10,7,-9999,195,205,220,235,250,260,300,295,285,290,295,300,300,310,315,-9999,2,2,2,1,3,4,12,7,3,6,7,10,11,7,7,1,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",4,15,8,1014,none,0,0,180,3,usa


In [75]:
# Borramos las columnas originales que como ya están fuera no hacen falta.

df_clima = df_clima.drop(columns=["rh_profile", "wind_profile"], axis= 1)

In [76]:
df_clima_medias = df_clima.groupby("country").mean().reset_index()

/tmp/ipykernel_1523/1277389351.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_clima_medias = df_clima.groupby("country").mean().reset_index()


In [77]:
df_clima_medias.head()

,country,wind_speed950mb,wind_speed900mb,wind_speed850mb,wind_speed800mb,wind_speed750mb,wind_speed700mb,wind_speed650mb,wind_speed600mb,wind_speed550mb,wind_speed500mb,wind_speed450mb,wind_speed400mb,wind_speed350mb,wind_speed300mb,wind_speed250mb,wind_speed200mb,wind_direction950mb,wind_direction900mb,wind_direction850mb,wind_direction800mb,wind_direction750mb,wind_direction700mb,wind_direction650mb,wind_direction600mb,wind_direction550mb,wind_direction500mb,wind_direction450mb,wind_direction400mb,wind_direction350mb,wind_direction300mb,wind_direction250mb,wind_direction200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,australia,3.12500,3.093750,3.312500,3.500000,3.687500,3.68750,3.390625,3.078125,2.796875,3.062500,3.265625,3.593750,4.328125,5.078125,5.562500,5.625000,92.421875,80.468750,82.812500,107.890625,127.031250,147.187500,151.953125,147.968750,176.093750,188.125000,201.875000,222.031250,238.437500,266.718750,266.718750,265.390625,3.390625,4.484375,6.015625,7.796875,9.734375,11.328125,11.531250,8.359375,4.406250,2.421875,-0.171875,-0.265625,-2.593750,-3.265625,-3.546875,-3.265625,31.5,97.5,4.468750,-9999.0,-9999.0,-9999.0,32.625000,-1.140625,3.187500,1005.531250,2.640625,0.00000,93.515625,2.734375
1,new zealand,-2654.46875,2.203125,2.296875,2.515625,2.906250,2.81250,2.750000,2.828125,2.968750,3.031250,3.093750,3.156250,3.359375,3.640625,3.921875,3.734375,-2558.171875,144.296875,155.703125,183.359375,170.234375,171.484375,163.359375,185.312500,185.546875,178.046875,177.890625,177.343750,181.250000,177.421875,184.375000,218.437500,-2647.906250,10.812500,11.843750,11.906250,10.015625,7.906250,5.812500,4.218750,3.718750,3.671875,3.734375,4.375000,5.406250,6.796875,7.265625,-1.390625,31.5,97.5,6.015625,-9999.0,-9999.0,-9999.0,15.046875,5.078125,11.109375,1015.578125,2.921875,0.00000,143.203125,1.953125
2,papua new guinea,-9999.00000,-9999.000000,-9999.000000,2.000000,2.000000,2.15625,2.390625,2.312500,2.718750,3.171875,3.593750,4.015625,4.265625,4.406250,4.859375,5.656250,-9999.000000,-9999.000000,-9999.000000,162.031250,197.656250,258.046875,244.687500,158.906250,74.062500,75.234375,80.390625,82.187500,92.421875,111.406250,114.375000,110.156250,-9999.000000,-9999.000000,-9999.000000,11.953125,12.484375,11.609375,11.156250,11.593750,11.578125,10.500000,9.593750,9.312500,11.468750,12.843750,12.375000,11.234375,31.5,97.5,8.218750,-9999.0,-9999.0,-9999.0,14.906250,0.015625,13.218750,-9999.000000,4.546875,0.00000,207.500000,1.984375
3,south africa,-9999.00000,-9999.000000,3.531250,3.125000,2.718750,2.50000,2.609375,2.843750,2.921875,3.046875,3.453125,3.609375,3.468750,3.218750,3.484375,3.875000,-9999.000000,-9999.000000,154.453125,220.781250,219.140625,248.437500,247.421875,235.078125,215.156250,237.421875,214.843750,239.765625,239.765625,253.828125,258.203125,269.531250,-9999.000000,-9999.000000,-0.046875,0.515625,1.500000,2.890625,5.062500,8.312500,11.359375,6.890625,0.468750,-0.562500,0.687500,3.500000,3.062500,5.390625,31.5,97.5,4.437500,-9999.0,-9999.0,-9999.0,31.640625,0.703125,0.140625,1010.062500,0.765625,0.00000,149.609375,3.015625
4,usa,-9999.00000,3.234375,3.437500,3.578125,3.890625,4.12500,4.328125,4.578125,4.875000,5.171875,5.515625,5.828125,6.406250,7.250000,8.187500,8.421875,-9999.000000,187.343750,199.453125,225.937500,234.765625,256.796875,254.453125,249.765625,254.218750,254.765625,271.718750,266.484375,264.843750,264.062500,273.593750,272.812500,-9999.000000,6.953125,4.953125,3.687500,4.312500,4.890625,5.234375,6.234375,6.250000,6.593750,6.703125,7.921875,9.578125,9.015625,6.593750,3.062500,31.5,97.5,5.921875,-9999.0,-9999.0,-9999.0,3.937500,13.218750,8.609375,1012.078125,0.796875,0.09375,190.625000,2.421875


Procedemos a unir los 2 DataFrames en uno.

In [78]:
df_total = df_tiburones_country.merge(df_clima_medias, on="country")
df_total.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM,wind_speed950mb,wind_speed900mb,wind_speed850mb,wind_speed800mb,wind_speed750mb,wind_speed700mb,wind_speed650mb,wind_speed600mb,wind_speed550mb,wind_speed500mb,wind_speed450mb,wind_speed400mb,wind_speed350mb,wind_speed300mb,wind_speed250mb,wind_speed200mb,wind_direction950mb,wind_direction900mb,wind_direction850mb,wind_direction800mb,wind_direction750mb,wind_direction700mb,wind_direction650mb,wind_direction600mb,wind_direction550mb,wind_direction500mb,wind_direction450mb,wind_direction400mb,wind_direction350mb,wind_direction300mb,wind_direction250mb,wind_direction200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327,-9999.0,3.234375,3.4375,3.578125,3.890625,4.125,4.328125,4.578125,4.875,5.171875,5.515625,5.828125,6.40625,7.25,8.1875,8.421875,-9999.0,187.34375,199.453125,225.9375,234.765625,256.796875,254.453125,249.765625,254.21875,254.765625,271.71875,266.484375,264.84375,264.0625,273.59375,272.8125,-9999.0,6.953125,4.953125,3.6875,4.3125,4.890625,5.234375,6.234375,6.25,6.59375,6.703125,7.921875,9.578125,9.015625,6.59375,3.0625,31.5,97.5,5.921875,-9999.0,-9999.0,-9999.0,3.9375,13.21875,8.609375,1012.078125,0.796875,0.09375,190.625,2.421875
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860,-9999.0,3.234375,3.4375,3.578125,3.890625,4.125,4.328125,4.578125,4.875,5.171875,5.515625,5.828125,6.40625,7.25,8.1875,8.421875,-9999.0,187.34375,199.453125,225.9375,234.765625,256.796875,254.453125,249.765625,254.21875,254.765625,271.71875,266.484375,264.84375,264.0625,273.59375,272.8125,-9999.0,6.953125,4.953125,3.6875,4.3125,4.890625,5.234375,6.234375,6.25,6.59375,6.703125,7.921875,9.578125,9.015625,6.59375,3.0625,31.5,97.5,5.921875,-9999.0,-9999.0,-9999.0,3.9375,13.21875,8.609375,1012.078125,0.796875,0.09375,190.625,2.421875


Guardar los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.

In [79]:
df_total.to_csv("datos/union_clima_medias_tiburones.csv")